# PyTorch Tutorial, Model (neural network) definition:
#### Torch.nn is used for model (neural network) definition 
- Download the MNIST dataset
- Convert the labels into one-hot vectors
- Use DataLoader on the downloaded MNIST dataset
- Define componenets of a neural network to deal with the MNIST
- Define the neural network with **Sequential** or in a *class* with **forward** method

https://github.com/ostad-ai/PyTorch-Tutorial

In [1]:
# importing the necessary modules
import torch
import torch.nn as tnn
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader

We download the MNIST dataset in both training_set and test_set, each contains pairs of (features,label). We have seen this in post of datasets

In [2]:
training_set=datasets.MNIST(root='./MNIST',train=True, download=True, transform=ToTensor())
test_set=datasets.MNIST(root='./MNIST',train=False,download=True,transform=ToTensor())

The **MNIST dataset** as shown below holds 60000 samples for training, and 10000 samples for testing.
Each sample is composed of an image with size 28-by-28, and label with integer value from set {0,1,2...,9}

In [3]:
print('Training dataset:')
print(f'Size of features: {training_set.data.size()}')
print(f'Size of labels: {training_set.targets.size()}')
print(f'The class labels are from set: {training_set.targets.unique()}')
Nclasses=training_set.targets.unique().size().numel()
print(f'The number of classes:{Nclasses}')
print('Test dataset:')
print(f'Size of features: {test_set.data.size()}')
print(f'Size of labels: {test_set.targets.size()}')

Training dataset:
Size of features: torch.Size([60000, 28, 28])
Size of labels: torch.Size([60000])
The class labels are from set: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
The number of classes:10
Test dataset:
Size of features: torch.Size([10000, 28, 28])
Size of labels: torch.Size([10000])


We saw in previous post that for classification problems, the labels of samples should be in form one-hot vectors.
<br>Therefore, we convert labels into one-hot vectors by assigning the **target_transform** of  datasets using one-hot-transform as defined below 

In [4]:
one_hot_transform=Lambda(lambda y:torch.zeros(Nclasses,dtype=torch.float).index_put([torch.tensor(y)],
                                                                         values=torch.tensor(1.)))
training_set.target_transform=one_hot_transform
test_set.target_transform=one_hot_transform

Checking that one-hot vectors are created by above target-transform:

In [5]:
index=torch.randint(len(training_set),(1,))[0].item()
_,y=training_set[index]
print(f'Label of sample {index} is: {training_set.targets[index]}')
print(f'One-hot vector of label above: {y}')

Label of sample 23664 is: 4
One-hot vector of label above: tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])


Now, we use **DataLoader** to make the dataset iterable

In [6]:
batchSize=8
train_dataloader=DataLoader(training_set,batch_size=batchSize,shuffle=True)
test_dataloader=DataLoader(test_set,batch_size=batchSize,shuffle=True)

Checking how a batch of dataset with dataloader is retreived:

In [7]:
# just one step in for-loop over train_dataloader
for x,y in train_dataloader:
    print(f'Size of a batch={batchSize} of features: {x.size()}')
    print(f'The number of channels of input features: {x.size()[1]}')
    print(f'Size of a batch={batchSize} of labels: {y.size()}')
    break

Size of a batch=8 of features: torch.Size([8, 1, 28, 28])
The number of channels of input features: 1
Size of a batch=8 of labels: torch.Size([8, 10])


For processing images of for (C,H,W), we may use convolutional layers. For example Conv2d(cin,cout,kernel_size,...)
<br> In the example below, a Conv2d layer is created of kernel-size=3, and cin=1, cout=32. Let's see how a batch of input pairs are changed in size

In [8]:
cin,cout,kernelSize=1,32,3
conv1=tnn.Conv2d(cin,cout,kernel_size=kernelSize)
out=conv1(x)
print(f'We have a batch={batchSize} of feature vectors having size: {list(x.size())}')
print(f'The conv2d output of a batch={batchSize} of feature vectors has the size: {list(x.size())}')

We have a batch=8 of feature vectors having size: [8, 1, 28, 28]
The conv2d output of a batch=8 of feature vectors has the size: [8, 1, 28, 28]


The convolutional layer has parameters we call them **weight** and **bias**. 
<br>Let's see the size of these paramters

In [9]:
print(f'Weight size of the conv2d layer: {list(conv1.weight.size())}')
print(f'Bias size of the conv2d layer: {conv1.bias.size().numel()}')

Weight size of the conv2d layer: [32, 1, 3, 3]
Bias size of the conv2d layer: 32


We usually use a maxpooling layer after a convolutional layer.
<br>We may use **MaxPool2d**, which is applied in the example below.

In [10]:
maxpool1=tnn.MaxPool2d(kernel_size=2,stride=2)
out2=maxpool1(out)
print(out2.size())

torch.Size([8, 32, 13, 13])


We may sometimes need to flatten the output of layer, especially a convolutional layer. 
<br>For this purpose, we may use **Flatten()**

In [11]:
flatten1=tnn.Flatten()
out3=flatten1(out2)
print(out3.size()) # which is 32*13*13

torch.Size([8, 5408])


A linear layer is composed of a number of neurons. For using such a layer, **Linear** is used. 

In [12]:
linear1=tnn.Linear(5408,Nclasses)
out4=linear1(out3)
print(out4.size())

torch.Size([8, 10])


In neural networks, we often use activation functions, which produce nonlienarity in the network.
<br> **ReLU** and **Softmax** are two examples of activation functions. These functions do not change the size of their inputs.

In [13]:
act1=tnn.Softmax(dim=1)
out5=act1(out4)
print(out5.size())

torch.Size([8, 10])


With the components introduced so far, we are able to define a neural network. 
<br>On way is to use **Sequential**. 
<br>The components of a *neural network* may be defined in *Sequential*, where the data is passed through them with the same order of components as defined.

In [14]:
model=tnn.Sequential(
      tnn.Conv2d(cin,cout,kernel_size=kernelSize),
      tnn.ReLU(),
      tnn.MaxPool2d(kernel_size=2,stride=2),
      tnn.Flatten(),
      tnn.Linear(5408,Nclasses),
      tnn.Softmax(dim=1)
)

The model gets the features part of each sample, and produces the output.
<br>Now, we see the size of a batch of input vectors, versus the size of a batch of output vectors.

In [15]:
y_out=model(x)
print(f'The size of input batch of features: {list(x.size())}')
print(f'The size of output batch: {list(y_out.size())}')

The size of input batch of features: [8, 1, 28, 28]
The size of output batch: [8, 10]


We can also define our neural netowk in a class inherited from torch.nn.Module. This class must have __init__() and forward() methods

In [16]:
class NNet(tnn.Module):
    def __init__(self,cin=1,cout=32,kernelSize=3,Nclasses=10):
        super().__init__()
        self.conv1=tnn.Conv2d(cin,cout,kernel_size=kernelSize)
        self.relu=tnn.ReLU()
        self.pool1=tnn.MaxPool2d(kernel_size=2,stride=2)
        self.flatten=tnn.Flatten()
        self.fc1=tnn.Linear(5408,Nclasses)
        self.softmax=tnn.Softmax(dim=1)
    def forward(self,x):
        return self.softmax(self.fc1(self.flatten(self.pool1(self.relu(self.conv1(x))))))

Applying the defined **NNet** to a batch of input vectors is the same as using **Sequential** mentioned earlier.

In [17]:
model2=NNet()
y_out2=model2(x)
print(f'The size of input batch of features: {list(x.size())}')
print(f'The size of output batch: {list(y_out2.size())}')

The size of input batch of features: [8, 1, 28, 28]
The size of output batch: [8, 10]
